In [ ]:
!pip install climate-learn

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import climate_learn as cl
from climate_learn.data.processing.nc2npz import convert_nc2npz
from IPython.display import HTML
import pytorch_lightning as pl
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar
)
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger
import torch

In [ ]:
# Download Base Data
cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/data/input/temperature",
    dataset="era5",
    variable="temperature_850"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/data/input/geopotential",
    dataset="era5",
    variable="geopotential_500"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/data/input/2m_temperature",
    dataset="era5",
    variable="2m_temperature"
)

# Download 2.8125 Data
cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/data/2.8125/input/2m_temperature",
    dataset="era5",
    variable="2m_temperature",
    resolution=2.8125  # optional, default is 5.625
)

# Download ERA5 Full Data Set
cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/temperature",
    dataset="era5",
    variable="temperature"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/geopotential",
    dataset="era5",
    variable="geopotential"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/2m_temperature",
    dataset="era5",
    variable="2m_temperature"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/10m_u_component_of_wind",
    dataset="era5",
    variable="10m_u_component_of_wind"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/10m_v_component_of_wind",
    dataset="era5",
    variable="10m_v_component_of_wind"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/constants",
    dataset="era5",
    variable="constants"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/relative_humidity",
    dataset="era5",
    variable="relative_humidity"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/specific_humidity",
    dataset="era5",
    variable="specific_humidity"
)

cl.data.download_weatherbench(
    dst="/content/drive/MyDrive/ClimateLearn/all_data/input/toa_incident_solar_radiation",
    dataset="era5",
    variable="toa_incident_solar_radiation"
)

In [ ]:
# convert base data set
convert_nc2npz(
    root_dir="/content/drive/MyDrive/ClimateLearn/data/input",
    save_dir="/content/drive/MyDrive/ClimateLearn/data/processed",
    variables=["temperature", "geopotential", "2m_temperature"],
    start_train_year=1979,
    start_val_year=2015,
    start_test_year=2017,
    end_year=2018,
    num_shards=16
)

# convert 2.8125 Data
convert_nc2npz(
    root_dir="/content/drive/MyDrive/ClimateLearn/data/2.8125/input",
    save_dir="/content/drive/MyDrive/ClimateLearn/data/2.8125/processed",
    variables=["2m_temperature"],
    start_train_year=1979,
    start_val_year=2015,
    start_test_year=2017,
    end_year=2018,
    num_shards=16
)

# convert ERA5 Full Dataset
convert_nc2npz(
    root_dir="/content/drive/MyDrive/ClimateLearn/all_data/input",
    save_dir="/content/drive/MyDrive/ClimateLearn/all_data/processed",
    variables=["temperature", "geopotential", "2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind",
               "constants", "relative_humidity", "specific_humidity", "toa_incident_solar_radiation"],
    start_train_year=1979,
    start_val_year=2015,
    start_test_year=2017,
    end_year=2018,
    num_shards=16
)


In [ ]:
#Forecasting Base models
dm = cl.data.IterDataModule(
    task="direct-forecasting",
    inp_root_dir="/content/drive/MyDrive/ClimateLearn/data/processed",
    out_root_dir="/content/drive/MyDrive/ClimateLearn/data/processed",
    in_vars=["temperature", "geopotential", "2m_temperature"],
    out_vars=["temperature", "geopotential", "2m_temperature"],
    src="era5",
    history=3,
    window=6,
    pred_range=120,
    subsample=6,
    batch_size=128,
    num_workers=8,
)
dm.setup()

climatology = cl.load_forecasting_module(
    data_module=dm, architecture="climatology"
)
persistence = cl.load_forecasting_module(
    data_module=dm, architecture="persistence"
)

trainer = pl.Trainer()
trainer.test(climatology, dm)
trainer.test(persistence, dm)

In [ ]:
#Forecasting Deep Learning models
gen_summary_depth = int(1)
gen_max_epochs=int(5)
gen_patience=int(5)
gen_gpu=int(-1)
gen_checkpoint=None

gen_forecast_type = "direct" #"iterative","continuous", "direct"
gen_model="resnet" # "unet", "vit", "resnet")
gen_pred_range= int(24) #6, 24, 72, 120, 240])
gen_era5_dir= "/content/drive/MyDrive/ClimateLearn/data/processed/"
# Set up data
variables = [
    "geopotential",
    "temperature",
    "u_component_of_wind",
    "v_component_of_wind",
    "relative_humidity",
    "specific_humidity",
    "2m_temperature",
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "toa_incident_solar_radiation",
    "land_sea_mask",
    "orography",
    "lattitude",
]
# PRESSURE_LEVEL_VARS = ["geopotential",
#     "temperature",
#     "u_component_of_wind",
#     "v_component_of_wind",
#     "relative_humidity",
#     "specific_humidity",]
# DEFAULT_PRESSURE_LEVELS = ["50", "250", "500", "600", "700", "850", "925"]
# in_vars = []
# for var in variables:
#     if var in PRESSURE_LEVEL_VARS:
#         for level in DEFAULT_PRESSURE_LEVELS:
#             in_vars.append(var + "_" + str(level))
#     else:
#         in_vars.append(var)
# if gen_forecast_type in ("direct", "continuous"):
#     out_variables = ["2m_temperature", "geopotential", "temperature"]
# elif gen_forecast_type == "iterative":
#     out_variables = variables
# out_vars = []
# for var in out_variables:
#     if var in PRESSURE_LEVEL_VARS:
#         for level in DEFAULT_PRESSURE_LEVELS:
#             out_vars.append(var + "_" + str(level))
#     else:
#         out_vars.append(var)
in_vars = ["2m_temperature", "geopotential", "temperature"]
out_vars = ["2m_temperature", "geopotential", "temperature"]
if gen_forecast_type in ("direct", "iterative"):
    dm = cl.data.IterDataModule(
        f"{gen_forecast_type}-forecasting",
        gen_era5_dir,
        gen_era5_dir,
        in_vars,
        out_vars,
        src="era5",
        history=3,
        window=6,
        pred_range=gen_pred_range,
        subsample=6,
        batch_size=128,
        num_workers=2,
    )
elif gen_forecast_type == "continuous":
    dm = cl.data.IterDataModule(
        "continuous-forecasting",
        gen_era5_dir,
        gen_era5_dir,
        in_vars,
        out_vars,
        src="era5",
        history=3,
        window=6,
        pred_range=1,
        max_pred_range=120,
        random_lead_time=True,
        hrs_each_step=1,
        subsample=6,
        batch_size=128,
        buffer_size=2000,
        num_workers=2,
    )
dm.setup()

# Set up deep learning model
in_channels = 3
if gen_forecast_type == "continuous":
    in_channels += 1  # time dimension
if gen_forecast_type == "iterative":  # iterative predicts every var
    out_channels = in_channels
else:
    out_channels = 3
if gen_model == "resnet":
    model_kwargs = {  # override some of the defaults
        "in_channels": in_channels,
        "out_channels": out_channels,
        "history": 3,
        "n_blocks": 28,
    }
elif gen_model == "unet":
    model_kwargs = {  # override some of the defaults
        "in_channels": in_channels,
        "out_channels": out_channels,
        "history": 3,
        "ch_mults": (1, 2, 2),
        "is_attn": (False, False, False),
    }
elif gen_model == "vit":
    model_kwargs = {  # override some of the defaults
        "img_size": (32, 64),
        "in_channels": in_channels,
        "out_channels": out_channels,
        "history": 3,
        "patch_size": 2,
        "embed_dim": 128,
        "depth": 8,
        "decoder_depth": 2,
        "learn_pos_emb": True,
        "num_heads": 4,
    }
optim_kwargs = {"lr": 5e-4, "weight_decay": 1e-5, "betas": (0.9, 0.99)}
sched_kwargs = {
    "warmup_epochs": 1,
    "max_epochs": 5,
    "warmup_start_lr": 1e-8,
    "eta_min": 1e-8,
}
model = cl.load_forecasting_module(
    data_module=dm,
    model=gen_model,
    model_kwargs=model_kwargs,
    optim="adamw",
    optim_kwargs=optim_kwargs,
    sched="linear-warmup-cosine-annealing",
    sched_kwargs=sched_kwargs,
)

# Setup trainer
pl.seed_everything(0)
default_root_dir = f"/content/drive/MyDrive/train_dir/forecasting/{gen_model}_{gen_forecast_type}_forecasting_{gen_pred_range}"
logger = TensorBoardLogger(save_dir=f"{default_root_dir}/logs")
early_stopping = "val/lat_mse:aggregate"
callbacks = [
    RichProgressBar(),
    RichModelSummary(max_depth=gen_summary_depth),
    EarlyStopping(monitor=early_stopping, patience=gen_patience),
    ModelCheckpoint(
        dirpath=f"{default_root_dir}/checkpoints",
        monitor=early_stopping,
        filename="epoch_{epoch:03d}",
        auto_insert_metric_name=False,
    ),
]
trainer = pl.Trainer(
    logger=logger,
    callbacks=callbacks,
    default_root_dir=default_root_dir,
    accelerator="gpu",
    devices=[0],
    max_epochs=gen_max_epochs,
    strategy="ddp_notebook",
    precision="16",
)


# Define testing regime for iterative forecasting
def iterative_testing(model, trainer, args, from_checkpoint=False):
    for lead_time in [6, 24, 72, 120, 240]:
        n_iters = lead_time // gen_pred_range
        model.set_mode("iter")
        model.set_n_iters(n_iters)
        test_dm = cl.data.IterDataModule(
            "iterative-forecasting",
            gen_era5_dir,
            gen_era5_dir,
            in_vars,
            out_vars,
            src="era5",
            history=3,
            window=6,
            pred_range=lead_time,
            subsample=1,
        )
        if from_checkpoint:
            trainer.test(model, datamodule=test_dm)
        else:
            trainer.test(model, datamodule=test_dm, ckpt_path="best")


# Define testing regime for continuous forecasting
def continuous_testing(model, trainer, args, from_checkpoint=False):
    for lead_time in [24, 72, 120]:
        test_dm = cl.data.IterDataModule(
            "continuous-forecasting",
            gen_era5_dir,
            gen_era5_dir,
            in_vars,
            out_vars,
            src="era5",
            history=3,
            window=6,
            pred_range=lead_time,
            max_pred_range=lead_time,
            random_lead_time=False,
            hrs_each_step=1,
            subsample=1,
            batch_size=128,
            buffer_size=2000,
            num_workers=8,
        )
        if from_checkpoint:
            trainer.test(model, datamodule=test_dm)
        else:
            trainer.test(model, datamodule=test_dm, ckpt_path="best")


# Train and evaluate model from scratch
if gen_checkpoint is None:
    trainer.fit(model, datamodule=dm)
    if gen_forecast_type == "direct":
        trainer.test(model, datamodule=dm, ckpt_path="best")
    elif gen_forecast_type == "iterative":
        iterative_testing(model, trainer, args)
    elif gen_forecast_type == "continuous":
        continuous_testing(model, trainer, args)
# Evaluate saved model checkpoint
else:
    model = cl.LitModule.load_from_checkpoint(
        gen_checkpoint,
        net=model.net,
        optimizer=model.optimizer,
        lr_scheduler=None,
        train_loss=None,
        val_loss=None,
        test_loss=model.test_loss,
        test_target_tranfsorms=model.test_target_transforms,
    )
    if gen_forecast_type == "direct":
        trainer.test(model, datamodule=dm)
    elif gen_forecast_type == "iterative":
        iterative_testing(model, trainer, args, from_checkpoint=True)
    elif gen_forecast_type == "continuous":
        continuous_testing(model, trainer, args, from_checkpoint=True)

In [ ]:
#Downscaling Base models
# Set up data
in_vars = out_vars = [
    "2m_temperature"]
dm = cl.data.IterDataModule(
    "downscaling",
    "/content/drive/MyDrive/ClimateLearn/data/processed/",
    "/content/drive/MyDrive/ClimateLearn/data/2.8125/processed",
    in_vars,
    out_vars,
    subsample=1,
    batch_size=32,
    num_workers=4,
)
dm.setup()

# Set up baseline models
nearest = cl.load_downscaling_module(data_module=dm, architecture="nearest-interpolation")
bilinear = cl.load_downscaling_module(data_module=dm, architecture="bilinear-interpolation")

# Evaluate baselines (no training needed)
trainer = pl.Trainer()
trainer.test(nearest, dm)
trainer.test(bilinear, dm)

In [ ]:
#Downscaling Deep learning models
low_res_dir ="/content/drive/MyDrive/ClimateLearn/data/processed/"
high_res_dir = "/content/drive/MyDrive/ClimateLearn/data/2.8125/processed"
gen_architecture = "vit" #["resnet", "unet", "vit"])

gen_summary_depth = int(1)
gen_max_epochs=int(5)
gen_patience=int(5)
gen_gpu=int(-1)
gen_checkpoint=None

# # Set up data
# variables = [
#     "land_sea_mask",
#     "orography",
#     "lattitude",
#     "toa_incident_solar_radiation",
#     "2m_temperature",
#     "10m_u_component_of_wind",
#     "10m_v_component_of_wind",
#     "geopotential",
#     "temperature",
#     "relative_humidity",
#     "specific_humidity",
#     "u_component_of_wind",
#     "v_component_of_wind",
# ]
# out_var_dict = {
#     "t2m": "2m_temperature",
#     "z500": "geopotential_500",
#     "t850": "temperature_850",
# }
# in_vars = []
# for var in variables:
#     if var in PRESSURE_LEVEL_VARS:
#         for level in DEFAULT_PRESSURE_LEVELS:
#             in_vars.append(var + "_" + str(level))
#     else:
#         in_vars.append(var)
in_vars = out_vars = ["2m_temperature"]

dm = cl.data.IterDataModule(
    "downscaling",
    low_res_dir,
    high_res_dir,
    in_vars,
    out_vars,
    subsample=6,
    batch_size=32,
    buffer_size=2000,
    num_workers=2,
)
dm.setup()

# # Set up deep learning model
model = cl.load_downscaling_module(data_module=dm, architecture=gen_architecture)

# Setup trainer
pl.seed_everything(0)
default_root_dir = f"/content/drive/MyDrive/train_dir/downscaling/{gen_architecture}_downscaling_2m_temperature"
logger = TensorBoardLogger(save_dir=f"{default_root_dir}/logs")
early_stopping = "val/mse:aggregate"
callbacks = [
    RichProgressBar(),
    RichModelSummary(max_depth=gen_summary_depth),
    EarlyStopping(monitor=early_stopping, patience=gen_patience),
    ModelCheckpoint(
        dirpath=f"{default_root_dir}/checkpoints",
        monitor=early_stopping,
        filename="epoch_{epoch:03d}",
        auto_insert_metric_name=False,
    ),
]
trainer = pl.Trainer(
    logger=logger,
    callbacks=callbacks,
    default_root_dir=default_root_dir,
    accelerator="gpu",
    devices=[0],
    max_epochs=gen_max_epochs,
    strategy="ddp_notebook",
    precision="16"
)

# Train and evaluate model from scratch
# if gen_checkpoint is None:
trainer.fit(model, datamodule=dm)
trainer.test(model, datamodule=dm, ckpt_path="best")
# Evaluate saved model checkpoint
# else:
#     model = cl.LitModule.load_from_checkpoint(
#         gen_checkpoint,
#         net=model.net,
#         optimizer=model.optimizer,
#         lr_scheduler=None,
#         train_loss=None,
#         val_loss=None,
#         test_loss=model.test_loss,
#         test_target_tranfsorms=model.test_target_transforms,
#     )
#     trainer.test(model, datamodule=dm)